In [1]:
from common import *
import smart_open

In [2]:
with open(join(DATA_FOLDER, 'gold_mongo.json'), 'r') as f:
    gold = json.load(f)

In [ ]:
def iter_docs(fnames):
    for i, fn in enumerate(fnames):
#         logging.info("%s: " % i + fn)
        with GzipFile(fn, 'rb') as f:
            docs = ujson.loads(f.read())
        for k,doc in docs.items():
            if k not in gold.keys():
                yield k,doc

def read_corpus(fnames, tokens_only=False): 
    for k, doc in iter_docs(fnames):
        unlisted = [w for s in doc for w in s]
        if tokens_only:
            yield unlisted
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(unlisted, [k])

In [ ]:
folder = join(DATA_FOLDER, 'documents/')
fnames = glob(join(folder, '*.json.gz'))

model = Doc2Vec(read_corpus(fnames), size=200, window=8, min_count=5, workers=cpu_count, iter=10)

2017-09-28 20:12:21,872 [MainThread  ] [DEBUG]  Fast version of gensim.models.word2vec is being used
2017-09-28 20:12:21,874 [MainThread  ] [INFO ]  collecting all words and their counts
2017-09-28 20:12:22,117 [MainThread  ] [INFO ]  PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-09-28 20:12:36,359 [MainThread  ] [INFO ]  PROGRESS: at example #10000, processed 18251055 words (1281669/s), 156491 word types, 10000 tags
2017-09-28 20:12:48,688 [MainThread  ] [INFO ]  PROGRESS: at example #20000, processed 32653756 words (1168302/s), 248872 word types, 20000 tags
2017-09-28 20:12:56,222 [MainThread  ] [INFO ]  PROGRESS: at example #30000, processed 41525327 words (1177819/s), 298791 word types, 30000 tags
2017-09-28 20:13:08,323 [MainThread  ] [INFO ]  PROGRESS: at example #40000, processed 54836204 words (1100095/s), 358164 word types, 40000 tags


In [ ]:
model.save('../data/saved/d2v_model_s200_w8_mc5_iter10')